In [1]:
%cd /content

repo_name = "VoxCPM-ja"
repo_url = "https://github.com/q9uri/VoxCPM-ja.git"
import os

if not os.path.exists(repo_name):
    !git clone {repo_url}
else:
    print(f"{repo_name} already exists. Pulling latest changes...")

%cd {repo_name}
!git pull

/content
Cloning into 'VoxCPM-ja'...
remote: Enumerating objects: 1497, done.
remote: Counting objects: 100% (225/225), done.
remote: Compressing objects: 100% (124/124), done.
remote: Total 1497 (delta 159), reused 110 (delta 101), pack-reused 1272 (from 1)
Receiving objects: 100% (1497/1497), 141.75 MiB | 14.95 MiB/s, done.
Resolving deltas: 100% (351/351), done.
Updating files: 100% (1763/1763), done.
/content/VoxCPM-ja
Already up to date.


In [11]:
with open("pyproject.toml", "r") as f:
    text = f.read()

text = text.replace('"jpreprocess",', "")

with open("pyproject.toml", "w") as f:
    text = f.write(text)

!pip install -e .

Obtaining file:///content/VoxCPM-ja
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached torchcodec-0.9.1-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (11 kB)
  Using cached gradio-6.5.1-py3-none-any.whl.metadata (16 kB)
  Using cached addict-2.4.0-py3-none-any.whl.metadata (1.0 kB)
  Using cached wetext-0.1.2-py3-none-any.whl.metadata (4.9 kB)
  Using cached modelscope-1.34.0-py3-none-any.whl.metadata (43 kB)
  Using cached datasets-3.6.0-py3-none-any.whl.metadata (19 kB)
  Using cached funasr-1.3.1-py3-none-any.whl.metadata (37 kB)
  Using cached spaces-0.47.0-py3-none-any.whl.metadata (1.0 kB)
  Using cached argbind-0.3.9.tar.gz (17 kB)
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.8/298.8 kB 12.2 MB/s eta 0:00:00
  Preparing metad

In [13]:
#patch g2p (colab and hf space don't install jpreprocess on 3.12)

!pip install pyopenjtalk-plus

text = """
import pyopenjtalk
from .lib.ja.voicevox_g2p.kana_converter import create_kana
from .lib.ja.voicevox_g2p.text_analyzer import full_context_labels_to_accent_phrases

def g2p(norm_text) -> str:
    full_context_labels = pyopenjtalk.extract_fullcontext(norm_text)
    accent_phrases = full_context_labels_to_accent_phrases(full_context_labels)
    kana = create_kana(accent_phrases)
    return kana
"""

with open("src/voxcpm/text/japanese.py", mode="w") as f:
  f.write(text)



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 17.9 MB/s eta 0:00:00


In [7]:
#download lora model
from huggingface_hub import hf_hub_download
import os

repo_id = "WariHima/1.5-large-ja-rev-a"
filename = ["lora_config.json", "lora_weights.safetensors", "optimizer.pth", "scheduler.pth"]

for file in filename:
  path = hf_hub_download(
      repo_id=repo_id,
      filename=file,
      local_dir="./checkpoints/1.5-large-ja-rev-a",
      local_dir_use_symlinks=False
  )
print(f"Downloaded to: {path}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_validators.py:202: UserWarning: The `local_dir_use_symlinks` argument is deprecated and ignored in `hf_hub_download`. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


lora_config.json:   0%|          | 0.00/493 [00:00<?, ?B/s]

lora_weights.safetensors:   0%|          | 0.00/65.6M [00:00<?, ?B/s]

optimizer.pth:   0%|          | 0.00/132M [00:00<?, ?B/s]

scheduler.pth:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

Downloaded to: checkpoints/1.5-large-ja-rev-a/scheduler.pth


In [8]:
#download asr model
from huggingface_hub import hf_hub_download
import os

repo_id = "FunAudioLLM/SenseVoiceSmall"
filename = ["am.mvn", "chn_jpn_yue_eng_ko_spectok.bpe.model", "config.yaml", "configuration.json", "model.pt"]

for file in filename:
  path = hf_hub_download(
      repo_id=repo_id,
      filename=file,
      local_dir="./checkpoints/SenseVoiceSmall",
      local_dir_use_symlinks=False
  )
print(f"Downloaded to: {path}")

am.mvn: 0.00B [00:00, ?B/s]

chn_jpn_yue_eng_ko_spectok.bpe.model:   0%|          | 0.00/377k [00:00<?, ?B/s]

config.yaml: 0.00B [00:00, ?B/s]

configuration.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

model.pt:   0%|          | 0.00/936M [00:00<?, ?B/s]

Downloaded to: checkpoints/SenseVoiceSmall/model.pt


In [ ]:
with open("app.py", "r") as f:
    text = f.read()

# share=True の置換
replace_list = [
    "interface.queue(max_size=10, default_concurrency_limit=1).launch(server_name=server_name, server_port=server_port, show_error=show_error)",
    "interface.queue(max_size=10, default_concurrency_limit=1).launch(server_name=server_name, server_port=server_port, show_error=show_error, share=True)",
]
text = text.replace(replace_list[0], replace_list[1])

with open("app.py", "w") as f:
    f.write(text)

#run gradio webui

!python app.py

🚀 Running on device: cuda
funasr version: 1.3.1.
* Running on local URL:  http://localhost:7860
* Running on public URL: https://3ba859dd2f38ed796f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
rtf_avg: 0.140: 100% 1/1 [00:01<00:00,  1.14s/it]
rtf_avg: 0.014: 100% 1/1 [00:00<00:00,  7.17it/s]
Model not loaded, initializing...
Using model dir: ./checkpoints/WariHima__voxcpm-1.5-resized-large
voxcpm_model_path: ./checkpoints/WariHima__voxcpm-1.5-resized-large, zipenhancer_model_path: iic/speech_zipenhancer_ans_multiloss_16k_base, enable_denoiser: True
Running on device: cuda, dtype: bfloat16
Loading model from safetensors: ./checkpoints/WariHima__voxcpm-1.5-resized-large/model.safetensors
Loading LoRA weights from: ./checkpoints/1.5-large-ja-rev-a
Loaded 640 LoRA parameters, skipped 0
2026-02-03 08:42:48,344 - model